In [1]:
import pandas as pd
import torch
from torchmetrics import Accuracy
import lightning.pytorch as pl
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader
from data import ResumeDataset, ResumePromptDataset

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print(torch.cuda.is_available()) 

True


In [3]:

from utils import label_idx, idx_label, all_labels, n_classes
target_json_path = "../data/json/data_fine.json"

df = pd.read_json(target_json_path)
print(df.iloc[1])


print(label_idx, idx_label, all_labels)

buf_str    (January 31, 2021)
lbuf                43.235294
rbuf                57.215686
hbuf                        1
boldbuf                     0
italbuf                     0
stk_str    James Hye Suk Yoon
lstk                41.098039
rstk                59.392157
hstk                        1
boldstk                     1
italstk                     0
type                  discard
Name: 1, dtype: object
{'discard': 0, 'merge': 1, 'pop': 2, 'subordinate': 3} {0: 'discard', 1: 'merge', 2: 'pop', 3: 'subordinate'} ['discard', 'merge', 'pop', 'subordinate']


In [4]:
tokenizer_args = {
    'padding': 'max_length',
    'return_tensors': 'pt',
}

In [5]:
from transformers import (
    AutoModel, AutoTokenizer, 
    T5ForConditionalGeneration,
    AdamW,
    get_linear_schedule_with_warmup
)

In [6]:
model_name = "t5-large"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
resume_dataset = ResumePromptDataset(tokenizer, target_json_path)
total_count = len(resume_dataset)
train_count = int(0.85 * total_count)
valid_count = int(0.1 * total_count)
test_count = total_count - train_count - valid_count

seed = torch.Generator().manual_seed(42)
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    resume_dataset, (train_count, valid_count, test_count)
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [63]:
batch = next(iter(valid_dataloader))

In [9]:
from experiment_textonly import T5FineTuner

# def get_dataset(tokenizer, type_path, args):
#   if(type_path == "train"):
#     return train_dataset
#   else:
#     return valid_dataset
  
# # class T5FineTuner(pl.LightningModule):
#   def __init__(self, hparams):
#     super(T5FineTuner, self).__init__()
#     self.hparams.update(vars(hparams))
    
#     self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
#     self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path)
#     self.save_hyperparameters()
#   def is_logger(self):
#     return self.trainer.proc_rank <= 0
  
#   def forward(
#       self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
#   ):
#     return self.model(
#         input_ids,
#         attention_mask=attention_mask,
#         decoder_input_ids=decoder_input_ids,
#         decoder_attention_mask=decoder_attention_mask,
#         labels=labels,
#     )

#   def _step(self, batch):
#     labels = batch["target_ids"]
#     labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

#     outputs = self(
#         input_ids=batch["source_ids"],
#         attention_mask=batch["source_mask"],
#         labels=labels,
#         decoder_attention_mask=batch['target_mask']
#     )

#     print(f"batch = {batch}, loss = {outputs[0]}")

#     loss = outputs[0]

#     return loss

#   def training_step(self, batch, batch_idx):
#     loss = self._step(batch)

#     tensorboard_logs = {"train_loss": loss}
#     self.log("Training Loss", loss)
#     return {"loss": loss, "log": tensorboard_logs}
  
#   def validation_step(self, batch, batch_idx):
#     loss = self._step(batch)
#     self.log("Validation Loss", loss)
#     return {"val_loss": loss}
  
#   def configure_optimizers(self):
#     "Prepare optimizer and schedule (linear warmup and decay)"

#     model = self.model
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {
#             "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": self.hparams.weight_decay,
#         },
#         {
#             "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#             "weight_decay": 0.0,
#         },
#     ]
#     optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
#     self.opt = optimizer
#     return [optimizer]
  
#   def get_tqdm_dict(self):
#     tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

#     return tqdm_dict

#   def train_dataloader(self):
#     train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
#     dataloader = DataLoader(train_dataset, drop_last=True, shuffle=True)
#     t_total = (
#         (len(dataloader.dataset) // (self.hparams.train_batch_size))
#         // self.hparams.gradient_accumulation_steps
#         * float(self.hparams.num_train_epochs)
#     )
#     scheduler = get_linear_schedule_with_warmup(
#         self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
#     )
#     self.lr_scheduler = scheduler
#     return dataloader

#   def val_dataloader(self):
#     val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
#     return DataLoader(val_dataset)

In [14]:
args_dict = dict(
    output_dir="~/sp23/sean-liu-resume-dependency/model/output",
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    tokenizer_args=tokenizer_args,
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=2,
    early_stop_callback=False,
    fp_16=False, # Setting this to true results in NaN values for the loss - this is widely documented unfortunately :(
    precision=16, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42
)

In [11]:
import argparse

args = argparse.Namespace(**args_dict)
print(args_dict)

{'output_dir': '~/sp23/sean-liu-resume-dependency/model/output', 'model_name_or_path': 't5-large', 'tokenizer_name_or_path': 't5-large', 'max_seq_length': 512, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 3, 'gradient_accumulation_steps': 2, 'early_stop_callback': False, 'fp_16': False, 'precision': 16, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


In [12]:
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    accelerator="gpu",
    devices = [0],
    gradient_clip_val=args.max_grad_norm,
    val_check_interval = 0.5 # validate every half epoch
)

In [53]:
model = T5FineTuner(args)

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [32]:
# for i in range(10):
#     batch = next(iter(train_dataloader))
#     loss = model._step(batch)
#     print(loss.item())
#     loss.backward()

In [51]:
from lightning.pytorch.tuner import Tuner
trainer = pl.Trainer(**train_params)
tuner = Tuner(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [52]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 737 M 
-----------------------------------------------------
737 M     Trainable params
0         Non-trainable params
737 M     Total params
2,950.672 Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]batch = {'source_ids': tensor([[ 2625,    10,  4406,     3,   117,  3388,  7765, 11430,     3, 16359,
         13383,     1,   931,    10, 27324,    10,  8014,  7986,    10,  1914,
          2783,    10,  3547,   769, 21122,    10,   220,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


batch = {'source_ids': tensor([[ 2625,    10,  1697, 26869,    23,  1612,  2166,    10, 14984,     7,
             6, 12702, 11393,     6,    11,    44,  1788,  1575,     5, 30090,
            26,  1350,  2569,    16,     8,     3,   117,  6741,  1744,  3040,
             7,  5841,    13,     8,  1775,    13,  1566,     6,  2808, 15505,
          1015,   636,     6,   932,  8013,     1,   931,    10, 27324,    10,
          8014,  7986,    10,  1914,  2783,    10,  3547,   769, 21122,    10,
           220,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/8104 [00:00<?, ?it/s] batch = {'source_ids': tensor([[ 2625,    10, 13017,    35,    53, 15077,   257,    16, 25274,  1356,
             6, 14383, 22172,     7,     6,     3, 31815,     3,   117,   868,
          2854,  2941,    41,  7369,   371,   201,  9871,   343,     5, 16574,
           447,     7,     6, 10107,  7959, 25274,     6, 18989,    11,     1,
           931,    10, 27324,    10,  8014,  7986,    10,  1914,  2783,    10,
          3547,   769, 21122,    10,   220,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0, 

/home/zxliu2/anaconda3/envs/fa23/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# Evaluation Testing 

In [13]:
model = T5FineTuner.load_from_checkpoint("/home/zxliu2/sp23/sean-liu-resume-dependency/model/output/epoch=2-step=12156.ckpt").eval()

AttributeError: 'Namespace' object has no attribute 'tokenizer_args'

In [ ]:
valid_dataloader = DataLoader(valid_dataset, num_workers=4)

source_ids source_mask target_ids target_mask


In [ ]:
batch = next(iter(valid_dataloader))
batch = {k: v.cuda() for k, v in batch.items()}

print(*batch)

source_ids source_mask target_ids target_mask


In [72]:
results = model._step(batch)

In [74]:
logits = results[0]
logits.shape

torch.Size([1, 32128])